In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models, datasets, transforms

In [5]:
# a conv block. it's convenient to use in multi times
def VGG_block(num_chan, in_channels, out_channels):
    layers = []
    for i in range(num_chan):
        layers.append(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                      kernel_size=3, padding=1)
        )
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    return nn.Sequential(*layers)


In [11]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

# VGG-11
def vgg(conv_arch):
    in_channels = 1
    layers = []
    for num_chan, output_channels in conv_arch:
        layers.append(VGG_block(num_chan=num_chan,
                      in_channels=in_channels, out_channels=output_channels))
        in_channels = output_channels
    return nn.Sequential(
        *layers, nn.Flatten(),
        # fc layers
        nn.Linear(output_channels*7*7, 4096), nn.ReLU(), nn.Dropout(.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10)
    )


In [12]:
net = vgg(conv_arch=conv_arch)

In [13]:
X = torch.randn(size=(1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__, 'output shape:\t', X.shape)


Sequential output shape:	 torch.Size([1, 64, 112, 112])
Sequential output shape:	 torch.Size([1, 128, 56, 56])
Sequential output shape:	 torch.Size([1, 256, 28, 28])
Sequential output shape:	 torch.Size([1, 512, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
Flatten output shape:	 torch.Size([1, 25088])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 10])
